<a href="https://colab.research.google.com/github/kanakesh2006/Langgraph_for_Agentic_AI_Concepts/blob/main/Persistence/joke_generator_persistence_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langgraph langchain_google_genai langchain_groq langchain_community langchain_core transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.1/719.1 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
 

In [4]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict, Annotated, Literal
from pydantic import BaseModel, Field
import operator

In [2]:
import os
from google.colab import userdata

# os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')


# using  GROQ's  "llama-3.1-8b-instant"  model
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')


# os.environ['HUGGINGFACE_API_KEY'] = userdata.get('HUGGINGFACE_API_KEY')


In [3]:
model = ChatGroq(model="llama-3.1-8b-instant")

In [5]:
# state

class JokeState(TypedDict):

  topic: str
  joke: str
  explanation: str

In [6]:
def generate_joke(state: JokeState):

  prompt = f'generate a joke on the topic - " {state["topic"]}" .'
  response = model.invoke(prompt).content

  return {'joke': response}


In [7]:
def generate_explanation(state: JokeState):

  prompt = f"write an explanation for the joke - '{state["joke"]}'. "
  response = model.invoke(prompt).content

  return {'explanation': response}


In [9]:
# graph
graph = StateGraph(JokeState)

# nodes
graph.add_node('Generate Joke', generate_joke)
graph.add_node('Generate Explanation', generate_explanation)

# edges
graph.add_edge(START, "Generate Joke")
graph.add_edge("Generate Joke", "Generate Explanation")
graph.add_edge("Generate Explanation", END)

# memory(persistence)
checkpointer = InMemorySaver()

# compile
workflow = graph.compile(checkpointer = checkpointer)


In [11]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':"pizza"}, config=config1)

{'topic': 'pizza',
 'joke': 'Why was the pizza in a bad mood? \n\nBecause it was feeling a little crusty!',
 'explanation': 'The joke "Why was the pizza in a bad mood? Because it was feeling a little crusty!" is an example of a play on words. \n\nIn this joke, the word \'crusty\' has a double meaning:\n\n1. A pizza crust is a hard, bread-like outer layer that surrounds the filling of a pizza.\n2. To be \'crusty\' can also mean being in a bad mood or having a gruff attitude.\n\nThe joke relies on this wordplay to create a pun, a form of humor that exploits multiple meanings of a word or phrase. The listener is expected to understand the double meaning of \'crusty\' and connect it to the context of the pizza, which creates the humorous effect.'}

In [12]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza in a bad mood? \n\nBecause it was feeling a little crusty!', 'explanation': 'The joke "Why was the pizza in a bad mood? Because it was feeling a little crusty!" is an example of a play on words. \n\nIn this joke, the word \'crusty\' has a double meaning:\n\n1. A pizza crust is a hard, bread-like outer layer that surrounds the filling of a pizza.\n2. To be \'crusty\' can also mean being in a bad mood or having a gruff attitude.\n\nThe joke relies on this wordplay to create a pun, a form of humor that exploits multiple meanings of a word or phrase. The listener is expected to understand the double meaning of \'crusty\' and connect it to the context of the pizza, which creates the humorous effect.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f555a-90f7-6701-8002-a346a35d8830'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-19T16:41:10.587327+

In [13]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza in a bad mood? \n\nBecause it was feeling a little crusty!', 'explanation': 'The joke "Why was the pizza in a bad mood? Because it was feeling a little crusty!" is an example of a play on words. \n\nIn this joke, the word \'crusty\' has a double meaning:\n\n1. A pizza crust is a hard, bread-like outer layer that surrounds the filling of a pizza.\n2. To be \'crusty\' can also mean being in a bad mood or having a gruff attitude.\n\nThe joke relies on this wordplay to create a pun, a form of humor that exploits multiple meanings of a word or phrase. The listener is expected to understand the double meaning of \'crusty\' and connect it to the context of the pizza, which creates the humorous effect.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f555a-90f7-6701-8002-a346a35d8830'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-19T16:41:10.587327

In [14]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti go to therapy? \n\nBecause it was feeling a little "twisted" and was struggling to "connect" with others.',
 'explanation': 'The joke relies on wordplay, using the double meaning of common phrases associated with both emotional struggles and the characteristics of spaghetti.\n\nThe phrase "feeling a little \'twisted\'" has a dual meaning: \n\n1. In a psychological context, it implies that the spaghetti is experiencing emotional distress or turmoil.\n2. In a culinary context, spaghetti is a type of long, thin, twisted pasta.\n\nSimilarly, the phrase "struggling to \'connect\' with others" also has a double meaning:\n\n1. In an emotional context, it suggests that the spaghetti is having difficulty forming relationships or bonds with others.\n2. In a culinary context, a plate of spaghetti is often served in a tangled, twisted mass, requiring the user to "connect" the different strands together with a fork.\n\nThe humor in this joke comes 

In [15]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti go to therapy? \n\nBecause it was feeling a little "twisted" and was struggling to "connect" with others.', 'explanation': 'The joke relies on wordplay, using the double meaning of common phrases associated with both emotional struggles and the characteristics of spaghetti.\n\nThe phrase "feeling a little \'twisted\'" has a dual meaning: \n\n1. In a psychological context, it implies that the spaghetti is experiencing emotional distress or turmoil.\n2. In a culinary context, spaghetti is a type of long, thin, twisted pasta.\n\nSimilarly, the phrase "struggling to \'connect\' with others" also has a double meaning:\n\n1. In an emotional context, it suggests that the spaghetti is having difficulty forming relationships or bonds with others.\n2. In a culinary context, a plate of spaghetti is often served in a tangled, twisted mass, requiring the user to "connect" the different strands together with a fork.\n\nThe humor 

In [16]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza in a bad mood? \n\nBecause it was feeling a little crusty!', 'explanation': 'The joke "Why was the pizza in a bad mood? Because it was feeling a little crusty!" is an example of a play on words. \n\nIn this joke, the word \'crusty\' has a double meaning:\n\n1. A pizza crust is a hard, bread-like outer layer that surrounds the filling of a pizza.\n2. To be \'crusty\' can also mean being in a bad mood or having a gruff attitude.\n\nThe joke relies on this wordplay to create a pun, a form of humor that exploits multiple meanings of a word or phrase. The listener is expected to understand the double meaning of \'crusty\' and connect it to the context of the pizza, which creates the humorous effect.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f555a-90f7-6701-8002-a346a35d8830'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-19T16:41:10.587327